# Detección de plagio utilizando bigramas y sentence embedding


## Introducción

En el trabajo "An Improved SRL based Plagiarism Detection Technique using Sentence Ranking" por Merin Paul y Sangeetha Jamal (1) se presenta un método de detección de plagio usando técnicas de Natural Language Processing (NLP) que consiste en:

1. Preprocesamiento
  * Segmentación del texto de entrada
  * Extracción de stopwords

2. Obtención de candidatos
  
  Obtención de los textos pertenecientes a un corpus que podrían haber sufrido plagio por parte del documento sospechoso. La misma se realiza utilizando el coeficiente de similitud de Jaccard.

3. Ranking de oraciones

  Selección de las oraciones con mayor posibilidad de contener plagio de un documento original. Dicha selección se obtiene realizando una vectorización sencilla de las oraciones y aplicando el método de similaridad coseno.
  
  La vectorización realizada consiste en tomar el conjunto S de términos pertenecientes al documento original de cardinalidad N y vectorizar cada oración como un vector (t1, t2, ..., tN); donde ti es igual a 1 si el término i está presente en la oración, 0 en otro caso.

  Luego, siendo A y B los vectores asociados a dos oraciones, la similaridad coseno se calcula de la siguiente forma:

  ![Similaridad Coseno](https://i.stack.imgur.com/Qmq2w.png)

  Cuanto mayor resulta el valor de la similaridad coseno, más similares son las oraciones y mayor posibilidad existe de que la oración B sea plagio de la oración A.

4. Semantic Role Labeling

  En NLP, realizar Semantic Role Labeling implica asignar a cada parte de una oración un propósito. El objetivo de la técnica es, a partir de una oración; determinar los fragmentos de la misma que responden a "qué", "quién", "donde", "cómo", "cuando", etc.
  El siguiente paso es, para los pares de oraciones que obtuvieron valores altos en la etapa anterior; aplicar Semantic Role Labeling, información que se utilizará en la etapa siguiente.

5. Detección de similaridad

  En esta última etapa se evalúa la similaridad de los pares de oraciónes obtenidos comparando los roles semánticos detectados.

En este trabajo, dadas las problematicas que implica aplicar Semantic Role Labeling (especialmente en el idioma español), implementaremos una simplificación de este método de detección de plagio.

## Método propuesto

A continuación, presentamos el método propuesto.

Las primeras dos etapas se mantienen iguales al método presentado por Paul y Jamal, siendo estas el preprocesamiento y la obtención de candidatos.

Una vez realizadas dichas etapas, se realiza el siguiente procedimiento.

1. Por cada oración perteneciente al documento sospechoso, se computa su sentence embedding.
2. Cada una de dichas oraciones se compara con las oraciones del documento original, computando el sentence embedding de la oración original y aplicando similaridad coseno entre los dos resultados.
3. Una oración se considera plagio cuando al menos una similaridad coseno con las oraciones de los documentos originales resultó alta.

### Cálculo de los sentence embeddings

Para el cómputo de los embeddings, se utiliza el modelo preentrenado [Google Universal Sentence Encoder](https://tfhub.dev/google/universal-sentence-encoder/3) en su versión multilenguaje V2. El mismo se encuentra entrenado con datos provenientes de 16 idiomas distíntos, incluido el español, y a probado tener buenos resultados en benchmarks de Semantic Similarity Retrieval.

## Implementación

In [0]:
# Instalación de dependencias

!pip3 install tensorflow_text>=2.0.0rc0

In [0]:
import tensorflow_hub as hub
import numpy as np
import nltk
from nltk import bigrams, word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
# Descarga de datos necesarios de nltk

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
# Obtención del modelo de vectorización de oraciones entrenado

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/2")

In [0]:
# Preprocesamiento

spanish_stopwords = stopwords.words('spanish')

def preprocess_document(text):
    return ' '.join([word.lower() for word in word_tokenize(text) if word.lower() not in spanish_stopwords])

In [0]:
# Obtención de candidatos

def jaccard_similarity_coefficient(suspect_n_grams, original_n_grams):
    return len(suspect_n_grams.intersection(original_n_grams)) / len(suspect_n_grams.union(original_n_grams))

def get_text_bygrams(text):
    return set(bigrams(word_tokenize(text)))

def may_be_plagiarism_of(suspect, original, threshold=0.2):
    preprocessed_original = preprocess_document(original)
    preprocessed_suspect = preprocess_document(suspect)    
    bigrams_original = get_text_bygrams(preprocessed_original)
    bigrams_suspect = get_text_bygrams(preprocessed_suspect)
    coefficient = jaccard_similarity_coefficient(
        bigrams_suspect, bigrams_original)
    return coefficient > threshold

In [0]:
# Detección de oraciones que incurrieron en plagio

def sentence_is_semantically_similar(suspect_sentence, original_sentence, threshold=0.7):
    suspect_embedding = embed(suspect_sentence)["outputs"].numpy().reshape(1, 512)
    original_embedding = embed(original_sentence)["outputs"].numpy().reshape(1, 512)
    return cosine_similarity(suspect_embedding, original_embedding) >= threshold

def any_match(collection, function):
    for element in collection:
        if function(element):
            return True
    return False

def sentence_is_plagiarism(suspect_sentence, original_document, cosine_similarity_threshold=0.7):
    return any_match(
        sent_tokenize(original_document),
        lambda original_sentence: sentence_is_semantically_similar(suspect_sentence, original_sentence, cosine_similarity_threshold)
    )

def plagiarised_sentences(suspect_document, original_document, cosine_similarity_threshold=0.7):
    suspect_sentences = sent_tokenize(suspect_document)
    return list(
        filter(
            lambda suspect_sentence: sentence_is_plagiarism(suspect_sentence, original_document, cosine_similarity_threshold),
            suspect_sentences
        )
    )

In [0]:
# Porcentaje de plagio en un documento

def plagiarism_percentage(suspect_document, original_document, cosine_similarity_threshold=0.7):
    n_plagiarised = len(plagiarised_sentences(suspect_document, original_document))
    n_total = len(sent_tokenize(suspect_document))
    return n_plagiarised / n_total

In [28]:
orig = "Hola soy Pablo. Me gusta la naranja."
plag = "Me gusta la naranja. Él se llama Pablo."
puede_ser_plagio = may_be_plagiarism_of(plag, orig)
porcentaje_de_plagio = plagiarism_percentage(plag, orig)
oraciones_plagio = plagiarised_sentences(plag, orig)
print(f"  - Puede ser plagio: {puede_ser_plagio}.")
print(f"  - Porcentaje de plagio: {porcentaje_de_plagio}.")
print(f"  - Oraciones que realizan plagio: {oraciones_plagio}")

  - Puede ser plagio: True.
  - Porcentaje de plagio: 0.5.
  - Oraciones que realizan plagio: ['Me gusta la naranja.']


## Próximos pasos

* Realizar pruebas exhaustivas del método y determinar los thresholds óptimos para el coeficiente de similaridad de Jaccard y la similaridad coseno.
* Implementar un sistema de Semantic Role Labeling en español y aplicar el método original propuesto por Paul y Jamal. Evaluar cómo se compara su rendimiento al método reducido.

## Referencias

1. Merin Paul, Sangeetha Jamal. 2014. An Improved SRL based Plagiarism Detection Technique using Sentence Ranking.
